In [ ]:
import numpy as np
from astropy.table import Table, join
from matplotlib import pyplot as plt

%matplotlib notebook


## Combine summary tables from each fit run

In [ ]:
all_ia = Table.read('sncosmo_results/snia_all_bands/summary.csv')
blue_ia = Table.read('sncosmo_results/snia_ug/summary.csv')
red_ia = Table.read('sncosmo_results/snia_riz/summary.csv')
all_91bg = Table.read('sncosmo_results/91bg_all_bands/summary.csv')
blue_91bg = Table.read('sncosmo_results/91bg_ug/summary.csv')
red_91bg = Table.read('sncosmo_results/91bg_riz/summary.csv')

tables = (blue_ia, red_ia, blue_91bg, red_91bg)
table_names = ('blue_ia', 'red_ia', 'blue_91bg', 'red_91bg')
combined_table = None
for table, name in zip(tables, table_names):
    # Rename columns
    for column in table.colnames:
        if column in ('cid', 'class'):
            continue
            
        table.rename_column(column, column + '_' + name)
        
    # Join tables
    if combined_table is None:
        combined_table = table
        continue
        
    combined_table = join(combined_table, table, 
                          join_type='outer', 
                          keys=['cid', 'class'])


## Chi-squared Comparison

In [ ]:
x_vals = combined_table['chi_blue_ia'] / combined_table['dof_blue_ia'] \
             - combined_table['chi_blue_91bg'] / combined_table['dof_blue_91bg']

y_vals = combined_table['chi_red_ia'] / combined_table['dof_red_ia'] \
             - combined_table['chi_red_91bg'] / combined_table['dof_red_91bg']


for classification in set(combined_table['class']):
    indx = combined_table['class'] == classification
    plt.scatter(x_vals[indx], y_vals[indx], s=5, label=classification)

plt.axvline(0, color='grey', alpha=0.5, linestyle='--')
plt.axhline(0, color='grey', alpha=0.5, linestyle='--')
plt.xlabel(r'$\chi^2_{ug}(Ia) - \chi^2_{ug}(91bg)$')
plt.ylabel(r'$\chi^2_{riz}(Ia) - \chi^2_{riz}(91bg)$')
plt.legend()
plt.show()
